## numpy相关的几个常数

numpy.Inf:浮点表示（正）无穷大。
numpy.NAN:浮点表示非数字（NaN）。NaN 和 NAN 是 nan 的等价定义。
上面两个常量都为浮点数。

## 一、numpy.NaN的特性

In [14]:
import numpy as np

* NaN在numpy的定义中是浮点数

In [2]:
data = np.random.randint(0,9,size=(3,5))
data

array([[3, 6, 2, 7, 3],
       [7, 7, 2, 3, 8],
       [7, 7, 4, 3, 2]])

In [5]:
#此处由于整个数组为整数(int)，因此np.NAN无法写入数组
data[0,1]=np.NAN  #nan,NaN

ValueError: cannot convert float NaN to integer

In [15]:
data = data.astype(np.float16)
data[0,1]=np.NaN
print(data)

[[ 3. nan  2.  7.  3.]
 [ 7.  7.  2.  3.  8.]
 [ 7.  7.  4.  3.  2.]]


* NaN和NaN是不相等的

In [12]:
np.NaN==np.NaN

False

* NaN和任何值运算都等于NaN

In [13]:
100*np.NaN

nan

## 二、如何去除数据中的NaN

In [29]:
data = np.random.randint(0,9,size=(3,5))
data=data.astype(np.float16)
data

array([[3., 1., 8., 4., 3.],
       [6., 2., 5., 4., 5.],
       [2., 0., 8., 7., 0.]], dtype=float16)

In [34]:
data[0,1]=np.nan
data[2,3]=np.nan
data

array([[ 3., nan,  8.,  4.,  3.],
       [ 6.,  2.,  5.,  4.,  5.],
       [ 2.,  0.,  8., nan,  0.]], dtype=float16)

### 1、isnan()函数，布尔数组简单提出nan数据

In [32]:
#使用isnan()函数生成布尔数组
np.isnan(data)

array([[False,  True, False, False, False],
       [False, False, False, False, False],
       [False, False, False,  True, False]])

In [33]:
#将nan数据直接剔除。但会形成一个一维数组，这样就会除掉原来的数组结构。
#此时实际上是针对原数组的切片操作，属于视图view,并不改变原数组。
data[~np.isnan(data)]

array([3., 8., 4., 3., 6., 2., 5., 4., 5., 2., 0., 8., 0.], dtype=float16)

### 2、np.delete()+where()函数，保留结构的函数删除nan数据

In [43]:
#delete精确删除数组中的行列数。乘以小数后自动变为float64数据类型。
data1=np.random.randint(0,9,size=(3,5))*1.35
data1

array([[ 0.  ,  2.7 ,  5.4 ,  1.35, 10.8 ],
       [ 1.35,  9.45,  6.75,  9.45,  9.45],
       [ 9.45,  1.35,  5.4 ,  5.4 ,  2.7 ]])

In [45]:
data1.dtype

dtype('float64')

In [47]:
data1[[1,2],[3,4]]=np.nan
data1

array([[ 0.  ,  2.7 ,  5.4 ,  1.35, 10.8 ],
       [ 1.35,  9.45,  6.75,   nan,  9.45],
       [ 9.45,  1.35,  5.4 ,  5.4 ,   nan]])

In [49]:
#用where函数找到nan位置，返回位置元组。元组1表示存在的行，元组2表示各列。
lines=np.where(np.isnan(data1))
lines

(array([1, 2]), array([3, 4]))

In [53]:
#delete()函数，整行整列删除。删除后不改变原始的数据。
np.delete(data1,lines[0],axis=0)

array([[ 0.  ,  2.7 ,  5.4 ,  1.35, 10.8 ]])

### 3、替换为某个数值

In [60]:
#默认读取数据时是自动转换成np.float格式。但存在空数据时，由于无法转换统一数据，那么会读取报错。
#存在空数据时可以首先以字符串的形式读取，再寻求其他的转换方法。
data2=np.loadtxt('nan.csv',delimiter=',',skiprows=1,dtype=np.str_)

In [64]:
data2[data2=='']=0.0
data2

array([['60', '88'],
       ['58', '0.0'],
       ['0.0', '98'],
       ['80', '69'],
       ['69', '79'],
       ['267', '334']], dtype='<U3')

In [67]:
data3=data2.astype(np.float64)
data3

array([[ 60.,  88.],
       [ 58.,   0.],
       [  0.,  98.],
       [ 80.,  69.],
       [ 69.,  79.],
       [267., 334.]])

### 3、求平均参数及总和的考虑

In [88]:
# 当设置dtype=str_时，默认为2U，字符串为'na'，此时不能转换为float的NaN。
# 将dtype='U25'，时能够在字符串和float之间的NAN转换。查阅文档可知字符串的具体类型。
data4=np.loadtxt('nan.csv',delimiter=',',skiprows=1,encoding='utf-8',dtype='U25')
data4[data4==""]=np.NAN
data4

array([['60', '88'],
       ['58', 'nan'],
       ['nan', '98'],
       ['80', '69'],
       ['69', '79'],
       ['52', '78']], dtype='<U25')

In [90]:
data5=data4.astype(np.float64)
data5

array([[60., 88.],
       [58., nan],
       [nan, 98.],
       [80., 69.],
       [69., 79.],
       [52., 78.]])

In [91]:
data5[np.isnan(data5)]=0
data5

array([[60., 88.],
       [58.,  0.],
       [ 0., 98.],
       [80., 69.],
       [69., 79.],
       [52., 78.]])

In [94]:
#参数axis的用法。delete函数axis-0表示行，之外的所有行数axis=1表示行。
math_sum=data5.sum(axis=1)
math_sum

array([148.,  58.,  98., 149., 148., 130.])

In [107]:
# 求平均值。此处出于统计的原因，不能直接将nan直接替换为0，否则平均值不准确。
# nan替换为除nan之外的平均值即可。在画图时比较有用。
data6=data4.astype(np.float64)
for x in range(data6.shape[1]):
    col = data6[:,x]
    #获取除nan值之外的数据的平均值
    #除XX之外在numpy中选取函数中表示为～
    mean = col[~np.isnan(col)].mean()
    print(mean)
    col[np.isnan(col)]=mean
data6

63.8
82.4


array([[60. , 88. ],
       [58. , 82.4],
       [63.8, 98. ],
       [80. , 69. ],
       [69. , 79. ],
       [52. , 78. ]])

In [100]:
data4.shape

(6, 2)